In [106]:
# Load dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, text
import pymc as pm
import arviz as az
from scipy.stats import zscore

# Define the SQLalchemy engine
engine = create_engine(f"sqlite:////Users/JO/PhD/neuro-ascertainment/data/db.sqlite")

# Read the SQL query from the file
with open('/Users/JO/PhD/neuro-ascertainment/candidate-queries/karolinska-nsicu-cohort/karolinska-cohort.sql', 'r') as file:
    query = file.read()

SEED = 20
rng = np.random.default_rng(SEED)

def count_id(df):
    LopNr = df['LopNr'].nunique() if 'LopNr' in df else 'Column missing'
    VtfId_LopNr = df['VtfId_LopNr'].nunique() if 'VtfId_LopNr' in df else 'Column missing'
    HADM_ID = df['HADM_ID'].nunique() if 'HADM_ID' in df else 'Column missing'
    return print(f'Unique patients: {LopNr} | Unique SIR admits: {VtfId_LopNr} | Unique PAR admits: {HADM_ID}')

### PAR_HADM
PAR_HADM contains all PAR admits:

In [107]:
query_PAR_HADM = query + "SELECT * FROM PAR_HADM"
PAR_HADM = pd.read_sql(query_PAR_HADM, engine)
count_id(PAR_HADM)

Unique patients: 59333 | Unique SIR admits: Column missing | Unique PAR admits: 359305


### K_ICU_ADMISSIONS
All ICU admissions at K CIVA and K NIVA

In [108]:
query_K_ICU_ADMISSIONS = query + "SELECT * FROM K_ICU_ADMISSIONS"
K_ICU_ADMISSIONS = pd.read_sql(query_K_ICU_ADMISSIONS, engine)
count_id(K_ICU_ADMISSIONS)

Unique patients: 6454 | Unique SIR admits: 7673 | Unique PAR admits: Column missing


### K_ICU_ADMISSIONS_MATCHED_WITH_PAR
Left join PAR admissions (with certain criteria) on K_ICU_ADMISSIONS on LopNr (patient ID) . Approx 800 patients are lost at this step as they do not have a proper patient ID ("reservnummer").

Note that each patient can have several ICU admits here. Also, each ICU admit can be matched with several PAR admits.

In [109]:
query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR = query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR"
K_ICU_ADMISSIONS_MATCHED_WITH_PAR = pd.read_sql(query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR, engine)
count_id(K_ICU_ADMISSIONS_MATCHED_WITH_PAR)

Unique patients: 5673 | Unique SIR admits: 6539 | Unique PAR admits: 7141


### K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX
Inferred diagnosis for the PAR admit is added to K_ICU_ADMISSIONS_MATCHED_WITH_PAR. In a few cases the criteria for several DX are fulfilled.

In [110]:
query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX = query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX"
K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX = pd.read_sql(query_K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX, engine)
count_id(K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX)

Unique patients: 5673 | Unique SIR admits: 6539 | Unique PAR admits: 7141


### K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX_TIME_HIERARCHY
This takes care of situations where a ICU admit is matched with several PAR admits. Here a column called DX_ORDER is introduced, ranking PAR admits based on "relevancy". The earliest admit is most relevant. If there is a tie on date, a diagnostic group hierarchy is used to decide rank. All rows with "OTHER" dx are removed, that's why we loose patients in this step.

In [111]:
query_TIME_HIERARCHY= query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX_TIME_HIERARCHY"
TIME_HIERARCHY  = pd.read_sql(query_TIME_HIERARCHY, engine)
count_id(TIME_HIERARCHY)

Unique patients: 4150 | Unique SIR admits: 4806 | Unique PAR admits: 4965


In [112]:
small_d = TIME_HIERARCHY[['VtfId_LopNr','InskrTidPunkt', 'UtskrTidPunkt', 'HADM_ID', 'MVO', 'LopNr', 'INDATUM', 'UTDATUM', 'DX_GROUP', 'DIAGNOS', 'DX_ORDER']]
small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
small_d['InskrTidPunkt'] = pd.to_datetime(small_d['InskrTidPunkt'], unit='s').dt.normalize()
small_d['UtskrTidPunkt'] = pd.to_datetime(small_d['UtskrTidPunkt'], unit='s').dt.normalize()

filtered_df = small_d[small_d.groupby('VtfId_LopNr')['VtfId_LopNr'].transform('size') > 1]
filtered_df['timediff_to_sir'] = filtered_df['INDATUM'] - filtered_df['InskrTidPunkt']

/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_21723/2561574566.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_21723/2561574566.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_21723/2561574566.py:4: SettingWithCopyWarning: 
A value is trying

In [113]:
check_vtf = rng.choice(filtered_df['VtfId_LopNr'], 1)
filtered_df.query(f"VtfId_LopNr == {check_vtf}")

,VtfId_LopNr,InskrTidPunkt,UtskrTidPunkt,HADM_ID,MVO,LopNr,INDATUM,UTDATUM,DX_GROUP,DIAGNOS,DX_ORDER,timediff_to_sir
5201,181018.0,2021-09-02,2021-10-05,117779,301,21159.0,2021-09-02,2021-09-03,TBI,S066 S066 S0630 S1210 S0210 S001 S0650 I620,1,0 days
5202,181018.0,2021-09-02,2021-10-05,117780,331,21159.0,2021-09-03,2021-10-05,TBI,S066 S066 S1210 S1210 S001 S001 S0650 S0650 S0...,2,1 days


### K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX_HIERARCHY_TIME
Same thing as above, but hierarchy first.

In [114]:
query_HIERARCHY_TIME= query + "SELECT * FROM K_ICU_ADMISSIONS_MATCHED_WITH_PAR_WITH_DX_HIERARCHY_TIME"
HIERARCHY_TIME  = pd.read_sql(query_HIERARCHY_TIME, engine)
count_id(HIERARCHY_TIME)

Unique patients: 4150 | Unique SIR admits: 4806 | Unique PAR admits: 4965


In [115]:
small_d = HIERARCHY_TIME[['VtfId_LopNr','InskrTidPunkt', 'UtskrTidPunkt', 'HADM_ID', 'MVO', 'LopNr', 'INDATUM', 'UTDATUM', 'DX_GROUP', 'DIAGNOS', 'DX_ORDER']]
small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
small_d['InskrTidPunkt'] = pd.to_datetime(small_d['InskrTidPunkt'], unit='s').dt.normalize()
small_d['UtskrTidPunkt'] = pd.to_datetime(small_d['UtskrTidPunkt'], unit='s').dt.normalize()

filtered_df = small_d[small_d.groupby('VtfId_LopNr')['VtfId_LopNr'].transform('size') > 1]
filtered_df['timediff_to_sir'] = filtered_df['INDATUM'] - filtered_df['InskrTidPunkt']

/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_21723/2839594063.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['INDATUM'] = pd.to_datetime(small_d['INDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_21723/2839594063.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_d['UTDATUM'] = pd.to_datetime(small_d['UTDATUM'], unit='D')
/var/folders/s_/hnbjtb757x10k0pyf6_x_9mw0000gn/T/ipykernel_21723/2839594063.py:4: SettingWithCopyWarning: 
A value is trying

In [116]:
check_vtf = rng.choice(filtered_df['VtfId_LopNr'], 1)
filtered_df.query(f"VtfId_LopNr == {check_vtf}")

,VtfId_LopNr,InskrTidPunkt,UtskrTidPunkt,HADM_ID,MVO,LopNr,INDATUM,UTDATUM,DX_GROUP,DIAGNOS,DX_ORDER,timediff_to_sir
1796,132411.0,2014-02-10,2014-02-14,68368,331,12135.0,2014-02-10,2014-02-16,ABM,G060 I489 I109 Z958 I340 J324,1,0 days
1797,132411.0,2014-02-10,2014-02-14,68370,121,12135.0,2014-02-20,2014-03-24,ABM,G060,2,10 days


### SUMMARY_TABLE
Finally, using the time and dx hierarchy as tiebreakers. We choose the first ICU admit for each patient and match it with a PAR admit.

In [117]:
query_DESC_PAR = query + "SELECT * FROM SUMMARY_TABLE"
DESC = pd.read_sql(query_DESC_PAR, engine)
DESC

DESC['par_adm_date'] = pd.to_datetime(DESC['par_adm_date'], unit='D')
DESC['par_dsc_date'] = pd.to_datetime(DESC['par_dsc_date'], unit='D')
DESC['sir_adm_time'] = pd.to_datetime(DESC['sir_adm_time'], unit='s').dt.normalize()
DESC['sir_dsc_time'] = pd.to_datetime(DESC['sir_dsc_time'], unit='s').dt.normalize()

In [118]:
count_id(DESC)

Unique patients: 4150 | Unique SIR admits: 4150 | Unique PAR admits: 4150


In [121]:
s = DESC.groupby('DX_GROUP').agg({'age': 'mean', 'sex_female': 'mean', 'sir_total_time': 'median', 'SAPS_GCS': 'median', 'any_AMV': 'mean', 'd30': 'mean', 'VtfId_LopNr': 'count'})
s.rename(columns={'VtfId_LopNr': 'n'}).sort_values(by='n', ascending=False)

,age,sex_female,sir_total_time,SAPS_GCS,any_AMV,d30,n
DX_GROUP,,,,,,,
ASAH,57.619849,0.645080,8436.0,14.0,0.637511,0.124474,1189
TBI,54.500000,0.256604,3000.0,11.0,0.790566,0.183019,1060
ICH,58.551127,0.407279,4143.0,8.0,0.856153,0.344887,577
AIS,63.350711,0.334123,2781.0,10.0,0.850711,0.281991,422
CFX,60.633858,0.255906,3502.5,15.0,0.700787,0.098425,254
ABM,54.734104,0.491329,4606.0,12.0,0.780347,0.121387,173
TUM,51.262295,0.442623,2944.5,11.0,0.762295,0.180328,122
SEP,55.666667,0.391667,1317.5,10.0,0.808333,0.016667,120
SDH,60.194444,0.305556,2805.0,12.0,0.712963,0.250000,108
